In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

In [2]:
# Define the reforms
vance_ref = Reform.from_dict({
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2025-01-01.2100-12-31": 5000
  },
  "gov.irs.credits.ctc.phase_out.amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.refundable.fully_refundable": {
    "2025-01-01.2100-12-31": True
  }
}, country_id="us")

vance_non_ref = Reform.from_dict({
  "gov.irs.credits.ctc.amount.base[0].amount": {
    "2025-01-01.2100-12-31": 5000
  },
  "gov.irs.credits.ctc.phase_out.amount": {
    "2025-01-01.2100-12-31": 0
  }
}, country_id="us")

reform_harris = Reform.from_dict({
  "gov.contrib.congress.delauro.american_family_act.baby_bonus": {
    "2025-01-01.2100-12-31": 2400
  },
  "gov.irs.credits.ctc.amount.arpa[0].amount": {
    "2025-01-01.2100-12-31": 3600
  },
  "gov.irs.credits.ctc.amount.arpa[1].amount": {
    "2025-01-01.2100-12-31": 3000
  },
  "gov.irs.credits.ctc.phase_out.arpa.in_effect": {
    "2025-01-01.2100-12-31": True
  },
  "gov.irs.credits.ctc.refundable.fully_refundable": {
    "2025-01-01.2100-12-31": True
  }
}, country_id="us")

In [3]:
# Function to run simulation and calculate budget impact
def calculate_budget_impact(reform=None, year=2024):
    baseline = Microsimulation(dataset="enhanced_cps_2024")
    baseline_budget = baseline.calculate("income_tax", year, map_to="household").sum() / 1e9

    if reform:
        reformed = Microsimulation(dataset="enhanced_cps_2024", reform=reform)
        reformed_budget = reformed.calculate("income_tax", year, map_to="household").sum() / 1e9
        return round(reformed_budget - baseline_budget, 1)
    else:
        return 0

In [4]:
# Initialize lists to store results
years = list(range(2025, 2035))
reforms = ["Baseline", "Vance (Refundable)", "Vance (Non-Refundable)", "Harris"]
results = {reform: [] for reform in reforms}


In [5]:
# Run simulations for each year and reform
for year in years:
    print(f"Processing year {year}...")
    results["Baseline"].append(0)
    results["Vance (Refundable)"].append(calculate_budget_impact(vance_ref, year))
    results["Vance (Non-Refundable)"].append(calculate_budget_impact(vance_non_ref, year))
    results["Harris"].append(calculate_budget_impact(reform_harris, year))



Processing year 2025...
Processing year 2026...
Processing year 2027...
Processing year 2028...
Processing year 2029...
Processing year 2030...
Processing year 2031...
Processing year 2032...
Processing year 2033...
Processing year 2034...


In [6]:
# Create DataFrame from results
df = pd.DataFrame(results, index=years)

# Display results
print(df)

# Save results to CSV
df.to_csv("ctc_reforms_budget_impact.csv")

print("Results saved to ctc_reforms_budget_impact.csv")

      Baseline  Vance (Refundable)  Vance (Non-Refundable)  Harris
2025         0              -234.8                  -129.0   -87.1
2026         0              -306.3                  -178.6  -197.6
2027         0              -309.3                  -183.0  -195.4
2028         0              -311.7                  -186.7  -192.8
2029         0              -314.2                  -190.5  -189.9
2030         0              -316.5                  -194.1  -187.1
2031         0              -318.7                  -197.3  -183.8
2032         0              -320.8                  -201.0  -180.6
2033         0              -322.9                  -204.5  -177.4
2034         0              -325.1                  -208.0  -174.3
Results saved to ctc_reforms_budget_impact.csv


In [7]:
import pandas as pd
import plotly.graph_objects as go
from policyengine_core.charts import *

In [9]:
# Read the CSV file
df = pd.read_csv("ctc_reforms_budget_impact.csv", index_col="Year")

# Multiply all reform values by -1 to make impact positive
for column in df.columns:
    if column != 'Year':  # Assuming 'Year' is not a column since it's the index
        df[column] = df[column] * -1


# Set up colors for the reforms
colors = {
    'Vance (Refundable)': '#18375f',  # Dark blue
    'Vance (Non-Refundable)': '#2976fe',  # Light blue
    'Harris': '#c5c5c5'  # Gray
}


In [12]:
# Create the horizontal bar chart
fig = go.Figure()

for reform in ['Vance (Refundable)', 'Vance (Non-Refundable)', 'Harris']:
    fig.add_trace(go.Bar(
        y=df.index,
        x=df[reform],
        name=reform,
        marker_color=colors[reform],
        orientation='h'  # This makes the bars horizontal
    ))

# Update layout
fig.update_layout(
    title='Budget Impact of CTC Reforms (2025-2034)',
    yaxis_title='Year',
    xaxis_title='Budget Impact (Billions $)',
    barmode='group',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    ),
    height=800,  # Increased height for better visibility
    width=1000,
    yaxis=dict(
        autorange="reversed"  # This reverses the y-axis to have 2025 at the top
    )
)

# Format x-axis to show dollar values
fig.update_xaxes(tickprefix='$', ticksuffix='B')

# Apply PolicyEngine formatting
fig = format_fig(fig)

# Show the figure
fig.show()